# Graphics Idea Auto Generator

1.⁠ ⁠ask openai to generate a graphic idea description, prompt engineered for the text to diagram engine \
2.⁠ ⁠⁠auto-generate the diagram

## Using OpenAI To Generate a Graphic Idea

In [1]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
from openai import AzureOpenAI
import openai
from docx2python import docx2python
import re
import textwrap
import fitz 

In [2]:
load_dotenv()

True

In [24]:
# OpenAI API configuration
openai.api_type = "azure"
openai.api_key = os.getenv("api_key_azure")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_version = os.getenv("api_version")
#openai_deployment = "sdgi-gpt-35-turbo-16k"
gpt_3_turbo = "gpt-3.5-turbo"
got_4o = "gpt-4o"

client = AzureOpenAI(
  api_key = os.getenv("api_key_azure"),  
  api_version = os.getenv("api_version"),
  azure_endpoint =os.getenv("AZURE_OPENAI_ENDPOINT") 
)

In [52]:
# use this function to make simple openAI Calls
def callOpenAI(messages, model):  
    response_entities = openai.chat.completions.create(
                    model=model,
                    messages=messages
                )
    response = response_entities.choices[0].message.content
    return response

## Text Extraction from Module Dataset

In [5]:
module_file_path = '../data/raw_data/SEA Module 3 Draft_v4.docx'
output_file = '../data/extracted_data'

In [6]:
def docx_text_cleaning(text):
    # Remove footage or image indications
    cleaned_text = re.sub(r'----[^-]+----', '', text)

    # Remove lines with only whitespace characters
    cleaned_text = cleaned_text.replace('\r', '\n')
    cleaned_text = re.sub(r"\n", " ", cleaned_text)  # remove newlines
    cleaned_text = re.sub(r"\s\s+", ' ', cleaned_text)
    cleaned_text = textwrap.fill(cleaned_text, width=80) 
    return cleaned_text

In [7]:
def extract_text_docx(file_path):
    with docx2python(file_path) as docx_content:
        raw_text = docx_content.text
        cleaned_text = docx_text_cleaning(raw_text)
        return cleaned_text

In [8]:
filename = 'SEA Module 3 Draft_v4'
content = extract_text_docx(module_file_path)
output_path = output_file + "/" + filename
with open(output_path, "w") as f:
        f.write(content)

In [48]:
def extract_text_by_toc(pdf_path):
    document = fitz.open(pdf_path)
    toc = document.get_toc()
    
    toc_text = {}

    for index, toc_item in enumerate(toc):
        _, title, start_page = toc_item
        if 'warp' in title.lower():
            break
        
        # Determine the end page
        if index + 1 < len(toc):
            _, _, next_start_page = toc[index + 1]
            end_page = next_start_page - 1
        else:
            end_page = len(document)

        # Extract text for the section
        text = ""
        for page_num in range(start_page - 1, end_page):
            text += document[page_num].get_text()

        # Store the text in the dictionary
        toc_text[title] = text

    return toc_text

In [49]:
# Usage example
pdf_path = "../data/raw_data/SEA Module 3 Draft_v4.pdf"
toc_text = extract_text_by_toc(pdf_path)
extracted_data = '../data/extracted_data'
for title, content in toc_text.items():
    output_path = output_file_path + "/" + title + ".txt"
    with open(output_path, "w") as f:
        f.write(content)

In [50]:
def generate_graphic_ides(text):
    model = "gpt-4"
    message = f"""My job is to create engaging visual graph for UNDP module content. Act like an expert graphic designer and provide a visual description for the graph that the graphic designer can then implement.    
    
    The content of the UNDP module is delimited $$$
    
    $$$
    {text}
    $$$
    """
    return callOpenAI(message, model)

In [ ]:
graphics_idea_folder = 'graphics_idea'
for filename in os.listdir(extracted_data)[:1]:
   with open(os.path.join(extracted_data, filename), 'r') as f:
       text = f.read()
       graphic_idea = generate_graphic_ides(text)
       output_path = graphics_idea_folder + "/" + title + ".txt"
       with open(output_path, "w") as f:
           f.write(content)

In [ ]:
def generate_graphics(text):
    model = "dall-e-3"
    message = f"""Create a graph figure for the UNDP module using the above visual description. The graph should be clean, easy to read, and effectively convey the intended information.
    
    ```
    {text}
    ```
    """
    return callOpenAI(message, model)

In [ ]:
for filename in os.listdir(graphics_idea_folder)[:1]:
   with open(os.path.join(graphics_idea_folder, filename), 'r') as f:
       text = f.read()
       graphic_idea = generate_graphic_ides(text)
       output_path = "graphics_vis/" + title + ".png"
       with open(output_path, "w") as f:
           f.write(content)